In [1]:
import os
os.chdir('F:\mldata\Devnagri Project\Train')
import cv2 
import os 
import numpy as np 
from random import shuffle 
from tqdm import tqdm 

In [ ]:
TRAIN_DIR = r'F:\mldata\Devnagri Project\Train'
TEST_DIR = r'F:\mldata\Devnagri Project\Test'

In [ ]:
def label_img(img): 
    img_label = img.split('.')[0]
    return img_label

In [ ]:
def create_train_data(): 
    # Creating an empty list where we should store the training data 
    # after a little preprocessing of the data 
    training_data = [] 
  
    # tqdm is only used for interactive loading 
    # loading the training data 
    for folders in os.listdir(TRAIN_DIR):
        path = os.path.join(TRAIN_DIR, folders)
        for img in os.listdir(path):
                img_path = os.path.join(path,img)
         
  
        # labeling the images 
                label = label_img(img) 
  
         
  
        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) 
  
        # resizing the image for processing them in the covnet 
                #img = cv2.resize(img, (28,28)) 
  
        # final step-forming the training data list with numpy array of the images 
                training_data.append([np.array(img.flatten()), np.array(folders)]) 
  
     
    return training_data 


In [ ]:
data=create_train_data()

In [ ]:
col=[]
for i in range(0,78200):
    col.append('row'+str(i))

In [ ]:

col_names=[]
for i in range(0,78200):
    col_names.append(data[i][1].tolist())

In [ ]:
name=pd.DataFrame(index=col,data=col_names)

In [ ]:
name.head()

In [ ]:
name=pd.DataFrame()
dfn=pd.DataFrame()
for i in range(0,78200):
    arr=data[i][0]
    arr_n=data[i][1]
    
    dfn['row'+str(i)]=arr

In [ ]:
dfn.head()

In [ ]:
result=dfn.transpose()

In [ ]:
result.shape

In [ ]:
result.head()

In [ ]:
name1=pd.DataFrame(index=col,data=result)

In [ ]:
name1.head()

In [ ]:
data = pd.merge(result, name1, how='inner', on = result.index)

In [ ]:
data=data.drop(columns=['key_0','Unnamed: 0'])


In [ ]:
data=data.rename(columns={'0_x':'Y','0_y':'0'})

In [ ]:
data.head()

In [ ]:
data.groupby("Y").count()

In [ ]:

import numpy as np
X = data.values[:,1:].astype(np.float64)
Y = data["Y"].values

In [ ]:
X=X/255.0

In [ ]:
import random
import numpy as np
import pandas as pd
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Keras Imports - CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [ ]:
del data
n_classes = 46

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

# Encode the categories
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [ ]:
img_height_rows = 32
img_width_cols = 32

In [ ]:
im_shape = (img_height_rows, img_width_cols, 1)
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)

In [ ]:
cnn = Sequential()

In [ ]:
kernelSize = (3, 3)
ip_activation = 'relu'
ip_conv_0 = Conv2D(filters=32, kernel_size=kernelSize, input_shape=im_shape, activation=ip_activation)
cnn.add(ip_conv_0)

In [ ]:
ip_conv_0_1 = Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation)
cnn.add(ip_conv_0_1)

# Add the Pooling layer
pool_0 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same")
cnn.add(pool_0)

In [ ]:
ip_conv_1 = Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation)
cnn.add(ip_conv_1)
ip_conv_1_1 = Conv2D(filters=64, kernel_size=kernelSize, activation=ip_activation)
cnn.add(ip_conv_1_1)

pool_1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same")
cnn.add(pool_1)

In [ ]:
drop_layer_0 = Dropout(0.2)
cnn.add(drop_layer_0)

In [ ]:
flat_layer_0 = Flatten()
cnn.add(Flatten())

In [ ]:
h_dense_0 = Dense(units=128, activation=ip_activation, kernel_initializer='uniform')
cnn.add(h_dense_0)
# Let's add one more before proceeding to the output layer
h_dense_1 = Dense(units=64, activation=ip_activation, kernel_initializer='uniform')
cnn.add(h_dense_1)

In [ ]:
op_activation = 'softmax'
output_layer = Dense(units=n_classes, activation=op_activation, kernel_initializer='uniform')
cnn.add(output_layer)

In [ ]:
opt = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
# Compile the classifier using the configuration we want
cnn.compile(optimizer=opt, loss=loss, metrics=metrics)

In [ ]:
print(cnn.summary())

In [ ]:
history = cnn.fit(x_train, y_train,
                  batch_size=32, epochs=10,
                  validation_data=(x_test, y_test))

In [ ]:
scores = cnn.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import pandas as pd
datatest=pd.read_csv('test_X.csv')

In [ ]:
datatest.head()

In [ ]:
import numpy as np
d = datatest.values[:]
d

In [ ]:
img_height_rows=32
img_width_cols=32
im_shape = (img_height_rows, img_width_cols, 1)
dt = d.reshape(d.shape[0], *im_shape)

In [ ]:
result=loaded_model.predict(dt)

In [ ]:
labels=[]
labels=np.unique(Y)

In [ ]:
label=[]
for i in range(0,len(result)-1):
    fl=labels[np.argmax(result[i])]
    label.append(fl)

In [ ]:
label

In [ ]:
predict_labels=pd.DataFrame(data={"predicted_labels": label })

In [ ]:
predict_labels.head()

In [ ]:
predict_labels.to_csv('pred.csv',index=False)